In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

In [3]:
sys.path.append(os.path.abspath(os.path.join("../../..")))

In [4]:
import pandas as pd

from notebooks.utils.constants import question_ids_map
from notebooks.utils.simplified_experiment import SimplifiedExperiment

/Users/david/eafit/eafit-tesis/.venv/lib/python3.11/site-packages/pinecone/data/index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


Retrieve answers file


In [5]:
train_dataset = pd.read_excel("../../aggregated_results/train_dataset.xlsx")
test_dataset = pd.read_excel("../../aggregated_results/test_dataset.xlsx")
answers_df_train = pd.read_excel(
    "../../../data/asc_842/lease_agreements_info/answers.xlsx"
)
answers_df_train = answers_df_train.iloc[train_dataset["Lease"] - 1]
answers_df_test = pd.read_excel(
    "../../../data/asc_842/lease_agreements_info/answers.xlsx"
)
answers_df_test = answers_df_test.iloc[test_dataset["Lease"] - 1]

Get questions to send to the AI


In [6]:
question_id = question_ids_map["end_date"]

In [7]:
template_string = """
Please review the lease contract details provided and answer the following question:

Question: According to the contract, when is the last day of the lease (i.e., the date on which the lessee will lose the right to use the assets)?

Instructions:
1. Locate the lease term clause: Start by finding the section in the contract that specifies the lease duration. This often includes both the start and end dates or the total length of the lease term.
2. Identify any specific end date: Check if a specific end date is stated within the lease term clause. If so, confirm that this date aligns with the intended duration of the lease agreement.
3. Consider any conditions for early termination or extension: Review whether the lease includes any clauses about early termination or extension that could alter the original end date. Only provide the last day of the lease as specified in the contract, excluding any optional extensions unless stated as part of the term.
4. Format the date: Once you have identified the end date, enter it in the `answer_date` field in the format YYYY-MM-DD.

Context: {context}
"""

## Variant 3

- gpt-4o-mini
- text-embedding-3-large
- structured outputs


In [8]:
experiment = SimplifiedExperiment(
    answers_df=answers_df_train,
    question_id=question_id,
    index_name="eafit-text-embedding-3-large",
    date_question=True,
    csv_results_filename="results/end_date_prompt_tuning_train_v3.csv",
    template_string=template_string,
    retriever_question="What are all the dates in this document?",
    use_llm_filter=True,
    numbers_list=[str(i).zfill(3) for i in (answers_df_train.index + 1).to_list()],
    chunks_filename="chunks/end_date_chunks_train_v3.txt",
)
experiment.run()

Lease: lease078
Answer: 2026-11-30
Real answer: 2026-12-01
INCORRECT
----------------------------------------------------
Lease: lease068
Answer: 2029-11-01
Real answer: 2029-11-01
CORRECT
----------------------------------------------------
Lease: lease076
Answer: 2023-06-28
Real answer: 2019-06-28
INCORRECT
----------------------------------------------------
Lease: lease022
Answer: 20__-__-__
Real answer: 2035-02-01
INCORRECT
----------------------------------------------------
Lease: lease092
Answer: 2023-02-25
Real answer: 2022-10-02
INCORRECT
----------------------------------------------------
Lease: lease099
Answer: 2025-01-31
Real answer: 2025-01-31
CORRECT
----------------------------------------------------
Lease: lease080
Answer: 2021-10-31
Real answer: 2021-10-31
CORRECT
----------------------------------------------------
Lease: lease084
Answer: 2016-05-01
Real answer: 2026-05-01
INCORRECT
----------------------------------------------------
Lease: lease039
Answer: 2026-1

In [9]:
experiment = SimplifiedExperiment(
    answers_df=answers_df_test,
    question_id=question_id,
    index_name="eafit-text-embedding-3-large",
    date_question=True,
    csv_results_filename="results/end_date_prompt_tuning_test_v3.csv",
    template_string=template_string,
    retriever_question="What are all the dates in this document?",
    use_llm_filter=True,
    numbers_list=[str(i).zfill(3) for i in (answers_df_test.index + 1).to_list()],
    chunks_filename="chunks/end_date_chunks_test_v3.txt",
)
experiment.run()

Lease: lease081
Answer: 2022-12-21
Real answer: 2022-12-21
CORRECT
----------------------------------------------------
Lease: lease060
Answer: 2027-03-31
Real answer: 2027-04-01
INCORRECT
----------------------------------------------------
Lease: lease048
Answer: 2027-10-31
Real answer: 2027-10-31
CORRECT
----------------------------------------------------
Lease: lease100
Answer: 2023-08-07
Real answer: 2023-08-07
CORRECT
----------------------------------------------------
Lease: lease010
Answer: 2023-02-01
Real answer: 2020-01-31
INCORRECT
----------------------------------------------------
Lease: lease033
Answer: 2023-01-31
Real answer: 2023-01-31
CORRECT
----------------------------------------------------
Lease: lease038
Answer: 2036-12-31
Real answer: 2036-12-01
INCORRECT
----------------------------------------------------
Lease: lease005
Answer: 2019-05-31
Real answer: 2005-03-31
INCORRECT
----------------------------------------------------
Lease: lease074
Answer: 2020-12-